In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
import plotly.graph_objects as go


In [3]:
from uuid import uuid4


In [4]:
import re

## Convert source to dataframe

In [5]:
df = pd.read_json('code_source.py')

Convert string date to datetime 

In [6]:
df['transaction_date'] = pd.to_datetime(df['transaction_date'] )

In [7]:
df['birthdate'] = pd.to_datetime(df['birthdate'] )

check latest date and farthest date of the dataframe

In [8]:
df['month'] = df['transaction_date'].dt.month_name()

In [9]:
df['month_num'] = df['transaction_date'].dt.month

In [10]:
df['transaction_date'].max()

Timestamp('2022-06-30 00:00:00')

In [11]:
df['transaction_date'].min()

Timestamp('2022-01-01 00:00:00')

In [12]:
# list_[1]

In [13]:
# inactive_list = df.groupby(['month_num']).username.apply(lambda x: ','.join(x)).fillna('').reset_index().rename({'username': 'previous_list'}, axis=1)

## Transform to Granularity

In [14]:
df = df.assign(var=df['transaction_items'].str.split(';'))

In [15]:
df = df.explode('var')

In [16]:
df['number_of_purchases'] = df['var'].str.split('(',expand=True)[1].str.replace('x','').str.replace(')','').astype(int)

C:\Users\kubales\AppData\Local\Temp\ipykernel_15664\3064442970.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['number_of_purchases'] = df['var'].str.split('(',expand=True)[1].str.replace('x','').str.replace(')','').astype(int)


In [17]:
df = df.assign(var=df['var'].str.split(','))

In [18]:
df = df.explode('var')

In [19]:
df = df[~df['var'].str.contains(f"\\(x")]

In [20]:
df['previous_month_num'] = df['month_num']-1

In [21]:
df.rename(columns ={'var': 'product_name'},inplace=True)

In [22]:
df

,address,birthdate,mail,name,sex,username,transaction_items,transaction_value,transaction_date,month,month_num,product_name,number_of_purchases,previous_month_num
0,USNS Wagner\nFPO AE 00917,1926-03-11,watsonbrenda@gmail.com,Jennifer Campbell,F,veronicawells,"Exotic Extras,Beef Chicharon,(x4)",5196,2022-01-12,January,1,Exotic Extras,4,0
0,USNS Wagner\nFPO AE 00917,1926-03-11,watsonbrenda@gmail.com,Jennifer Campbell,F,veronicawells,"Exotic Extras,Beef Chicharon,(x4)",5196,2022-01-12,January,1,Beef Chicharon,4,0
1,USCGC Benson\nFPO AP 48794,2016-07-08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"Exotic Extras,Beef Chicharon,(x3);HealthyKid 3...",12056,2022-01-12,January,1,Exotic Extras,3,0
1,USCGC Benson\nFPO AP 48794,2016-07-08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"Exotic Extras,Beef Chicharon,(x3);HealthyKid 3...",12056,2022-01-12,January,1,Beef Chicharon,3,0
1,USCGC Benson\nFPO AP 48794,2016-07-08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"Exotic Extras,Beef Chicharon,(x3);HealthyKid 3...",12056,2022-01-12,January,1,HealthyKid 3+,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83035,"PSC 1089, Box 3720\nAPO AE 08973",2012-04-30,xjones@hotmail.com,Stephanie Russell,F,cmcdowell,"HealthyKid 3+,Nutrional Milk,(x1);Candy City,G...",2639,2022-06-03,June,6,Nutrional Milk,1,5
83035,"PSC 1089, Box 3720\nAPO AE 08973",2012-04-30,xjones@hotmail.com,Stephanie Russell,F,cmcdowell,"HealthyKid 3+,Nutrional Milk,(x1);Candy City,G...",2639,2022-06-03,June,6,Candy City,3,5
83035,"PSC 1089, Box 3720\nAPO AE 08973",2012-04-30,xjones@hotmail.com,Stephanie Russell,F,cmcdowell,"HealthyKid 3+,Nutrional Milk,(x1);Candy City,G...",2639,2022-06-03,June,6,Gummy Worms,3,5
83035,"PSC 1089, Box 3720\nAPO AE 08973",2012-04-30,xjones@hotmail.com,Stephanie Russell,F,cmcdowell,"HealthyKid 3+,Nutrional Milk,(x1);Candy City,G...",2639,2022-06-03,June,6,Candy City,1,5


### Number of Purchases by Product

In [23]:
df_by_product = df.groupby(['month_num','month','product_name'])['number_of_purchases'].sum().reset_index().sort_values(['month_num','number_of_purchases'],axis=0,ascending= False)

In [24]:
print(df_by_product)

    month_num     month        product_name  number_of_purchases
55          6      June       HealthyKid 3+                29469
51          6      June          Candy City                20040
52          6      June       Exotic Extras                20006
58          6      June        Orange Beans                10106
56          6      June  Kimchi and Seaweed                10104
53          6      June      Gummy Vitamins                 9980
54          6      June         Gummy Worms                 9934
50          6      June      Beef Chicharon                 9902
57          6      June      Nutrional Milk                 9767
59          6      June    Yummy Vegetables                 9722
45          5       May       HealthyKid 3+                29564
42          5       May       Exotic Extras                19801
41          5       May          Candy City                19765
40          5       May      Beef Chicharon                10028
48          5       May  

In [25]:
monthly_count_purchases = df_by_product.groupby(['month_num'])['number_of_purchases'].sum().reset_index()

In [26]:
monthly_count_purchases

,month_num,number_of_purchases
0,1,136082
1,2,139820
2,3,138594
3,4,138514
4,5,138260
5,6,139030


In [27]:
monthly_numbers = df.drop_duplicates(subset = ['transaction_date','username'])

In [28]:
monthly_numbers = monthly_numbers.groupby(['month','month_num'])['transaction_value'].sum().reset_index().sort_values(['month_num','transaction_value'],ascending=False)

In [29]:
monthly_numbers = monthly_numbers.merge( monthly_count_purchases, how='left', on='month_num')

In [30]:
monthly_numbers['transaction_value'] = monthly_numbers['transaction_value'].astype(float)

# Monthly Numbers

In [31]:
monthly_numbers

,month,month_num,transaction_value,number_of_purchases
0,June,6,63541471.0,139030
1,May,5,63541199.0,138260
2,April,4,63219566.0,138514
3,March,3,63733860.0,138594
4,February,2,63668196.0,139820
5,January,1,62234587.0,136082


## Monthly Repeater

In [32]:
previous_month_list = df.groupby(['month_num']).username.apply(lambda x: ','.join(x)).shift(1).fillna('').reset_index().rename({'username': 'previous_list'}, axis=1)
repeater = pd.merge(df, previous_month_list, how='left', on='month_num')
repeater['repeaters'] =repeater.apply(lambda row: row['username'] in row['previous_list'], axis=1)


In [40]:
repeater = repeater.drop_duplicates(subset = ['month','username'])

In [43]:
repeater['repeater_count'] = np.select([repeater['repeaters'] == True],[1],default=0)

C:\Users\kubales\AppData\Local\Temp\ipykernel_15664\161375157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repeater['repeater_count'] = np.select([repeater['repeaters'] == True],[1],default=0)


In [45]:
repeater = repeater.groupby(['month_num'])['repeater_count'].sum().reset_index()

In [46]:
repeater

,month_num,repeater_count
0,1,0
1,2,5194
2,3,5276
3,4,5229
4,5,5167
5,6,5246


## Monthly Active numbers

In [33]:
df_active_engaged = df.drop_duplicates(subset = ['month','username'])

In [34]:
df_active_engaged['count'] = 1

C:\Users\kubales\AppData\Local\Temp\ipykernel_15664\3237565106.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_active_engaged['count'] = 1


In [35]:
frame = []
for x in df_active_engaged['month_num'].unique():
    df_ = df_active_engaged[df_active_engaged['month_num']<= x]
    df_ = df_.groupby(['username']).sum().reset_index()
    df_['month_num'] = x
#     df_ = df[['month_num','username','count']]
    frame.append(df_)
                       
    

In [36]:
df_active_engaged = pd.concat(frame)

In [37]:
df_active_engaged = df_active_engaged[['month_num','username','count']]

In [49]:
df_active_engaged['engaged'] = np.select([(df_active_engaged['month_num'] - df_active_engaged['count']== 0) & (df_active_engaged['month_num'] != 1)],[1],default=0)

In [51]:
df_active_engaged = df_active_engaged.groupby(['month_num'])['engaged'].sum().reset_index()

In [52]:
df_active_engaged

,month_num,engaged
0,1,0
1,2,5167
2,3,4141
3,4,3304
4,5,2685
5,6,2210


## Inactive List

In [65]:
df_inactive = df.drop_duplicates(subset = ['month_num','username'])

In [66]:
df_inactive = df_inactive[['month_num','username']]

In [93]:
inactive_frame = []
for x in df_inactive['month_num'].unique():
    
    df_a = df_inactive[df_inactive['month_num'] == x]
    df_i = df_inactive[df_inactive['month_num']< x]
    df_i = df_i.drop_duplicates(subset = 'username')
    df_i = df_i[~df_i['username'].isin(df_a['username'])]
    df_i = df_i['username'].count()
    df_a = df_a.groupby(['month_num']).count().reset_index()
    df_a['inactive'] = df_i
    df_a['month_num_'] = x

    
#     df_ = df[['month_num','username','count']]
    inactive_frame.append(df_a)
                       

In [95]:
inactive_frame = pd.concat(inactive_frame)

In [97]:
inactive_frame = inactive_frame[['month_num_','inactive','username']]

In [100]:
inactive_frame.rename(columns ={'month_num_': 'month_num','username':'monthly_customer_count'},inplace=True)

C:\Users\kubales\AppData\Local\Temp\ipykernel_15664\3833968428.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inactive_frame.rename(columns ={'month_num_': 'month_num','username':'monthly_customer_count'},inplace=True)


# Summary

In [127]:
summary = monthly_numbers.merge(repeater,on='month_num',how='left')

In [128]:
summary = summary.merge(df_active_engaged,on='month_num',how='left')

In [129]:
summary = summary.merge(inactive_frame,on='month_num',how='left')

In [130]:
summary = summary.sort_values(['month_num'],ascending=True)

In [120]:
import plotly.graph_objects as go


In [131]:
summary = summary[['month', 'transaction_value', 'number_of_purchases',

                   'repeater_count', 'engaged', 'inactive', 'monthly_customer_count']]

In [137]:
import plotly.figure_factory as ff


In [140]:
summary.columns

Index(['month', 'transaction_value', 'number_of_purchases', 'repeater_count',
       'engaged', 'inactive', 'monthly_customer_count'],
      dtype='object')

In [144]:
summary

,month,transaction_value,number_of_purchases,repeater_count,engaged,inactive,monthly_customer_count
5,January,62234587.0,136082,0,0,0,6583
4,February,63668196.0,139820,5194,5167,1416,6630
3,March,63733860.0,138594,5276,4141,1728,6658
2,April,63219566.0,138514,5229,3304,1912,6568
1,May,63541199.0,138260,5167,2685,1915,6587
0,June,63541471.0,139030,5246,2210,1840,6665


In [221]:
rowEvenColor = 'lightgrey'
rowOddColor = 'white'


fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"}],
           [{"type": "scatter"}],
           [{"type": "scatter"}]]
)

fig = go.Figure(data=[go.Table(

    header=dict(values=['Month','Total Amount','# of Purchases','Repeater','Engaged','Inactive','# of Customers'],
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[summary.month, summary.transaction_value, summary.number_of_purchases, summary.repeater_count,
                      summary.engaged,summary.inactive,summary.monthly_customer_count,],
              #  fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*10],
               fill_color='lavender',
               align='left',
              height = 60))
],
              )





fig.update_layout(
    title_text = 'Digi Foods Monthly Summary Analysis',
    height = 800,
    margin = {'t':75, 'l':50},
    yaxis = {'domain': [0, .45]},
    xaxis2 = {'anchor': 'y2'},
    yaxis2 = {'domain': [.6, 1], 'anchor': 'x2', 'title': 'Goals'}
)


fig.show()

In [220]:
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.03,
    specs=[[{"type": "table"}],
           [{"type": "scatter"}],
           [{"type": "scatter"}]]
)


fig.add_trace(go.Bar(x=summary.month, y=summary.repeater_count, xaxis='x2', yaxis='y2',
                marker=dict(color='#0099ff'),
                name='Previous<br>Month<br>#'))

fig.add_trace(go.Bar(x=summary.month, y=summary.inactive, xaxis='x2', yaxis='y2',
                marker=dict(color='#40ffff'),
                name='Inactive<br>#'))

fig.add_trace(go.Bar(x=summary.month, y=summary.engaged, xaxis='x2', yaxis='y2',
                marker=dict(color='#0030ff'),
                name='Active<br>All<br>Months'))
fig.add_trace(go.Bar(x=summary.month, y=summary.monthly_customer_count, xaxis='x2', yaxis='y2',
                marker=dict(color='#003fff'),
                name='Current<br>Active<br>Customers'))


fig.update_layout(
    title_text = 'Digi Foods Customer Analysis',
    height = 800,
    margin = {'t':100, 'l':50},
    yaxis = {'domain': [0, .45]},
    xaxis2 = {'anchor': 'y2'},
    yaxis2 = {'domain': [.6, 1], 'anchor': 'x2', 'title': 'Number Of Customers'}
)


fig.show()

In [119]:
summary

,month,month_num,transaction_value,number_of_purchases,repeater_count,engaged,inactive,monthly_customer_count
5,January,1,62234587.0,136082,0,0,0,6583
4,February,2,63668196.0,139820,5194,5167,1416,6630
3,March,3,63733860.0,138594,5276,4141,1728,6658
2,April,4,63219566.0,138514,5229,3304,1912,6568
1,May,5,63541199.0,138260,5167,2685,1915,6587
0,June,6,63541471.0,139030,5246,2210,1840,6665


In [223]:
df_by_product = df_by_product[['month','product_name','number_of_purchases']]

In [226]:
df_by_product.rename(columns={'month':'Month','product_name':"Product Name","number_of_purchases":"Number Of Purchases"},inplace=True)

In [227]:
fig = ff.create_table(df_by_product)
fig.show()

,month_num,month,product_name,number_of_purchases
55,6,June,HealthyKid 3+,29469
51,6,June,Candy City,20040
52,6,June,Exotic Extras,20006
58,6,June,Orange Beans,10106
56,6,June,Kimchi and Seaweed,10104
53,6,June,Gummy Vitamins,9980
54,6,June,Gummy Worms,9934
50,6,June,Beef Chicharon,9902
57,6,June,Nutrional Milk,9767
59,6,June,Yummy Vegetables,9722
